In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

In [3]:
!mkdir -p drive
!google-drive-ocamlfuse drive
# !google-drive-ocamlfuse -o nonempty drive
# # # !google-drive-ocamlfuse datalab
# # import os
# # os.chdir(drive/AI/STEALTH)
# # # # os.getcwd()


datalab  drive
[Errno 2] No such file or directory: 'drive/AI/STEALTH/day4'
/content


In [8]:
!ls 
# !os.chdir('drive')
%cd day4_v2

Copy_of_Copy_of_ASHIS_BATCH_3_ASSIGNMENT4B_78_39.ipynb	day4_v1  day4_v2
/content/drive/STEALTH/day4/day4_v2


In [9]:
!pwd

/content/drive/STEALTH/day4/day4_v2


In [10]:
import keras
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.8
dropout_rate = 0.2

In [12]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height)
print(img_width)
print(channel)
# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 37s 0us/step
32
32
3


In [13]:
print(x_train.shape)
print(x_test.shape)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [20]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
activation_64 (Activation)      (None, 2, 2, 108)    0           batch_normalization_64[0][0]     
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, 2, 2, 9)      8748        activation_64[0][0]              
__________________________________________________________________________________________________
dropout_64 (Dropout)            (None, 2, 2, 9)      0           conv2d_65[0][0]                  
__________________________________________________________________________________________________
concatenate_60 (Concatenate)    (None, 2, 2, 117)    0           concatenate_59[0][0]             
                                                                 dropout_64[0][0]                 
__________________________________________________________________________________________________
batch_norm

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [22]:
!pwd

/content/drive/STEALTH/day4/day4_v2


In [0]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpoint= ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=3)
callbacks_list = [checkpoint]

In [0]:
# x_train.shape[0]
# datagen = ImageDataGenerator(zoom_range=0.25, horizontal_flip=True)

# datagen.fit(x_train, augment=True)

In [0]:

# model_info = model.fit_generator(datagen.flow(x_train, y_train, batch_size = 128),
#                                  steps_per_epoch = x_train.shape[0], nb_epoch = 30, 
#                                  validation_data = (x_test, y_test), verbose=1,callbacks=callbacks_list)

In [0]:
# datagen.shape[0]
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 239s 5ms/step - loss: 1.7751 - acc: 0.3175 - val_loss: 2.4125 - val_acc: 0.2999
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:09 - loss: 1.4529 - acc: 0.457450000/50000 [==============================] - 222s 4ms/step - loss: 1.3785 - acc: 0.4919 - val_loss: 1.5839 - val_acc: 0.4417
Epoch 3/50
45568/50000 [==========================>...] - ETA: 18s - loss: 1.1968 - acc: 0.568650000/50000 [==============================] - 222s 4ms/step - loss: 1.1904 - acc: 0.5717 - val_loss: 1.3611 - val_acc: 0.5251

Epoch 00003: saving model to weights-improvement-03-0.53.hdf5
Epoch 4/50
43136/50000 [========================>.....] - ETA: 28s - loss: 1.0830 - acc: 0.610650000/50000 [==============================] - 223s 4ms/step - loss: 1.0769 - acc: 0.6130 - val_loss: 1.3957 - val_acc: 0.5335
Epoch 5/50
50000/50000 [==============================] - 222s 4ms/step - loss: 0.

 1024/50000 [..............................] - ETA: 3:27 - loss: 0.4078 - acc: 0.865250000/50000 [==============================] - 222s 4ms/step - loss: 0.4128 - acc: 0.8574 - val_loss: 0.6046 - val_acc: 0.8203
Epoch 41/50
38400/50000 [======================>.......] - ETA: 48s - loss: 0.4041 - acc: 0.860450000/50000 [==============================] - 223s 4ms/step - loss: 0.4083 - acc: 0.8591 - val_loss: 0.7531 - val_acc: 0.7908
Epoch 42/50
50000/50000 [==============================] - 223s 4ms/step - loss: 0.4077 - acc: 0.8583 - val_loss: 0.7353 - val_acc: 0.7953

Epoch 00042: saving model to weights-improvement-42-0.80.hdf5
Epoch 43/50
 1024/50000 [..............................] - ETA: 3:32 - loss: 0.3939 - acc: 0.858450000/50000 [==============================] - 223s 4ms/step - loss: 0.4041 - acc: 0.8593 - val_loss: 0.7384 - val_acc: 0.7939
Epoch 44/50
38400/50000 [======================>.......] - ETA: 48s - loss: 0.3945 - acc: 0.863050000/50000 [==============================

In [0]:
# Due to network issue , my model fitting got stopped at 48th epoch , hence I went to last callback i.e-46th epoch and loaded the weight from 46th epoch
# and then train the model again for 6 epochs. And arrive at an accuracy of 83.18. Please check below.
model.load_weights("weights-improvement-45-0.79.hdf5")


In [28]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=6,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 50000 samples, validate on 10000 samples
Epoch 1/6
50000/50000 [==============================] - 223s 4ms/step - loss: 0.3922 - acc: 0.8646 - val_loss: 0.7084 - val_acc: 0.7960

Epoch 00001: saving model to weights-improvement-01-0.80.hdf5
Epoch 2/6
15744/50000 [========>.....................] - ETA: 2:25 - loss: 0.3849 - acc: 0.867450000/50000 [==============================] - 223s 4ms/step - loss: 0.3845 - acc: 0.8666 - val_loss: 0.6192 - val_acc: 0.8161
Epoch 3/6
44416/50000 [=========================>....] - ETA: 23s - loss: 0.3802 - acc: 0.868850000/50000 [==============================] - 223s 4ms/step - loss: 0.3819 - acc: 0.8686 - val_loss: 0.8099 - val_acc: 0.7882
Epoch 4/6
50000/50000 [==============================] - 223s 4ms/step - loss: 0.3772 - acc: 0.8680 - val_loss: 0.7763 - val_acc: 0.7754

Epoch 00004: saving model to weights-improvement-04-0.78.hdf5
Epoch 5/6
 2560/50000 [>.............................] - ETA: 3:20 - loss: 0.3837 - acc: 0.866850000/50000 

In [29]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 19s 2ms/step
Test loss: 0.5990854527950287
Test accuracy: 0.8318


In [30]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_83.18.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model_83.18.h5')

In [0]:
 model.load_weights('DNST_model_83.18.h5')